In [1]:
# common packages
import pandas as pd
import time

# # DL framework
import torch
from torchtext import data

from attractivedata import AttractiveData
from trainer import AttractiveTrainer

## Load and prepare data

In [2]:
train_file = 'data/train.csv'
test_file = 'data/test.csv'
pretrained_file = 'glove.840B.300d'
config = {
    'max_size': 48,
    'min_freq': 5,
    'batch_size': 64,
    'pretrained_file': pretrained_file
}


In [3]:
AttractiveData = AttractiveData(train_file, test_file, pretrained_file, config)

In [4]:
for i, sentence in enumerate(AttractiveData.test_data):
    if i == 3:
        print(vars(AttractiveData.train_data[i]), vars(sentence))

{'Headline': ['Sorry', ',', 'i', 'spent', 'it', 'on', 'myself', '!', 'Harvey', 'Nichols', "'", 'hilarious', 'Christmas', 'advert', 'sees', 'people', 'treating', 'themselves', 'instead', 'of', 'others'], 'Category': 'femail', 'Label': '0.18333333333333313'} {'Headline': ['Three', 'police', 'officers', 'accused', 'of', 'stealing', '?', '?', '30k', 'during', 'raid', 'on', 'criminal'], 'Category': 'news'}


In [5]:
len(AttractiveData.CATEGORIES_LABEL.vocab.freqs)

18

In [6]:
max_len = 0
a = AttractiveData.train_data
for i in range(len(a)):
    if len(a[i].Headline) >= max_len:
        max_len = len(a[i].Headline)
max_len

45

## Start to train

In [7]:
num_workers = 10

config['timestr'] = time.strftime("%Y%m%d-%H%M%S")
config['save_name'] = 'LSTM'
config['input_dim'] = len(AttractiveData.TEXT.vocab)
config['embedding_dim'] = 300
config['category_dim'] = len(AttractiveData.CATEGORIES_LABEL.vocab)
config['category_embedding_dim'] = 16
config['hidden_dim'] = 128
config['output_dim'] = 1
config['log_steps'] = 10
config['epochs'] = 20
config['lr'] = {
    'encoder': 3e-5,
    'embedding': 1e-5,
    'linear': 1e-4
}
config['num_layers'] = 1
config['nhead'] = 4
config['kernel_size'] = 3
config['dropout'] = 0.1

pretrained_embeddings = AttractiveData.TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([1518, 300])


In [8]:
AttractiveTrainer = AttractiveTrainer(config, AttractiveData.device, AttractiveData.trainloader, pretrained_embeddings)

In [9]:
AttractiveTrainer.model, AttractiveTrainer.config['total_params'], AttractiveTrainer.config['total_learned_params']

(AttractiveNet(
   (embedding): AttractiveEmbedding(
     (token): TokenEmbedding(1518, 300, padding_idx=1)
   )
   (encoder): LSTM(300, 128, dropout=0.1, bidirectional=True)
   (linear_output): Linear(in_features=256, out_features=1, bias=True)
 ),
 895977,
 440577)

In [10]:
AttractiveTrainer.train()

Epoch:   5%|▌         | 1/20 [00:00<00:05,  3.68it/s]
EP_train | avg_loss: 0.5334173962473869 |
Epoch:  10%|█         | 2/20 [00:00<00:04,  3.85it/s]
EP_train | avg_loss: 0.5322569962590933 |
Epoch:  15%|█▌        | 3/20 [00:00<00:04,  4.01it/s]
EP_train | avg_loss: 0.5322300931438804 |
Epoch:  20%|██        | 4/20 [00:00<00:03,  4.18it/s]
EP_train | avg_loss: 0.532259282656014 |
Epoch:  25%|██▌       | 5/20 [00:01<00:03,  4.20it/s]
EP_train | avg_loss: 0.5320576746016741 |
Epoch:  30%|███       | 6/20 [00:01<00:03,  4.15it/s]
EP_train | avg_loss: 0.5327534331008792 |
Epoch:  35%|███▌      | 7/20 [00:01<00:03,  4.25it/s]
EP_train | avg_loss: 0.5323472563177347 |
Epoch:  40%|████      | 8/20 [00:01<00:02,  4.33it/s]
EP_train | avg_loss: 0.5331293186172843 |
Epoch:  45%|████▌     | 9/20 [00:02<00:02,  4.39it/s]
EP_train | avg_loss: 0.5324353026226163 |
Epoch:  50%|█████     | 10/20 [00:02<00:02,  4.43it/s]
EP_train | avg_loss: 0.5323125496506691 |
Epoch:  55%|█████▌    | 11/20 [00:02<00:

## for classification, not better

In [14]:
# from sklearn.metrics import mean_squared_error
# a = AttractiveTrainer.train_predict
# AttractiveData.LABEL.vocab.itos[int(a[0])], AttractiveTrainer.train_true[0]
# correct = 0
# pred_list = []
# true_list = []
# for i in range(len(a)):
#     pred = AttractiveData.LABEL.vocab.itos[int(a[i])]
#     pred_list.append(float(pred))
#     true = AttractiveData.LABEL.vocab.itos[int(AttractiveTrainer.train_true[i])]
#     true_list.append(float(true))
# mean_squared_error(true_list, pred_list)
# # true_list

0.5601443355119825

## Below is testing

In [11]:
from attractivenet import AttractiveNet
PATH = './model/LSTM_20201031-194214_0.5323.20'
# load_model = TransformerModel(config).to(AttractiveData.device)
load_model = AttractiveNet(config).to(AttractiveData.device)
load_model.load_state_dict(torch.load(PATH))
load_model.eval()

AttractiveNet(
  (embedding): AttractiveEmbedding(
    (token): TokenEmbedding(1518, 300, padding_idx=1)
  )
  (encoder): LSTM(300, 128, dropout=0.1, bidirectional=True)
  (linear_output): Linear(in_features=256, out_features=1, bias=True)
)

In [12]:
def predict_attractive(sentence, category):
    if len(sentence) < config['max_size']:
        sentence += ['0'] * (config['max_size'] - len(sentence))
    else:
        sentence = sentence[:config['max_size']]
    indexed_sentence = [AttractiveData.TEXT.vocab.stoi[t] for t in sentence]
    indexed_category = [AttractiveData.CATEGORIES_LABEL.vocab.stoi[category]]
    tensor_sentence = torch.LongTensor(indexed_sentence).to(AttractiveData.device)
    tensor_category = torch.LongTensor(indexed_category).to(AttractiveData.device)

    tensor_sentence = tensor_sentence.unsqueeze(1)

    prediction = load_model(tensor_sentence, tensor_category)
    
    return prediction

In [13]:
# train mean = 3.15, test mean = 2.8
predict_list = []
for i, sentence in enumerate(AttractiveData.test_data):
    prediction = predict_attractive(sentence.Headline, sentence.Category)
    predict_list.append(prediction.item() + 2.8)
    # predict_list.append(prediction.item())
AttractiveData.df_test['Label'] = predict_list
AttractiveData.df_test[['ID', 'Label']].to_csv(config['save_name'] + '.csv', index=False)

In [14]:
# train_category = list(AttractiveData.CATEGORIES_LABEL.vocab.freqs)
# test_category = list(AttractiveData.df_test['Category'].value_counts().keys())
# for each_test in test_category:
#     if each_test not in train_category:
#         print(each_test)
# print()
# for each_train in train_category:
#     if each_train not in test_category:
#         print(each_train)

## Below just for fun guess

In [15]:
import statistics
from sklearn.metrics import mean_squared_error

In [16]:
# train mean = 3.15, test mean = 2.8
train_list = []
for i, sentence in enumerate(AttractiveData.train_data):
    prediction = predict_attractive(sentence.Headline, sentence.Category)
    train_list.append(prediction.item() + 3.15)
    # train_list.append(prediction.item())
mean_squared_error(pd.read_csv('data/train.csv').sort_values(['ID']).Label.to_list(), train_list), statistics.mean(train_list), statistics.stdev(train_list)

(0.5275844079046633, 3.144413550989246, 0.01213976555398827)

In [17]:
a = AttractiveData.df_train['Label'].to_list()
statistics.mean(a), statistics.stdev(a)

(0.0004084967320261136, 0.7295015193216009)

In [18]:
statistics.mean(predict_list), statistics.stdev(predict_list)

(2.7907568613239584, 0.013064209406311998)

In [19]:
baseline_list = pd.read_csv('baseline.csv').sort_values(['ID']).Label.to_list()
mean_squared_error(baseline_list, predict_list), statistics.mean(baseline_list), statistics.stdev(baseline_list)

(0.08929560530168232, 2.7156126782757597, 0.29355123275379763)

In [14]:
# LSTM my best
# statistics.mean(predict_list), statistics.stdev(predict_list)

(2.8167915543795683, 0.14611407210842048)